# Neighborhoods in Toronto

## PART I
### Put the data from wikipedia into a pandas dataframe 
### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [68]:
import pandas as pd

toronto_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

df = pd.DataFrame(data = toronto_data[0])

df.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [69]:
df = df[df.Borough != 'Not assigned']

In [70]:
df.head(12)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### More than one neighborhood can exist in one postal code area. The two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [71]:
pd_new = pd.DataFrame(df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x))).reset_index()

In [72]:
pd_new.head(12)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. Only one field is updated (Queen's Park)

In [73]:
pd_new[pd_new.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [74]:
pd_new.loc[pd_new.Borough == "Queen's Park", 'Neighbourhood'] = "Queen's Park"

In [75]:
pd_new[pd_new.Neighbourhood == "Queen's Park"]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


### The shape method print the number of rows of your dataframe.

In [76]:
pd_new.shape

(103, 3)

# PART II

### Add Latitude & Longitude to Dataframe (with geocoder installation)

In [77]:
#!conda install -c conda-forge geocoder --yes

### ATTENTION!: geocoder has created reliability problems.I used this csv online http://cocl.us/Geospatial_data'¶

In [78]:
df_GEO = pd.read_csv('http://cocl.us/Geospatial_data')
df_GEO.head(12)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [79]:
pd_new['Latitude'] = '0';
pd_new['Longitude'] = '0';

for i in pd_new.index:
    for j in df_GEO.index:
        if pd_new.iloc[i, 0] == df_GEO.iloc[j, 0]:
            pd_new.iloc[i, 3] = df_GEO.iloc[j, 1]
            pd_new.iloc[i, 4] = df_GEO.iloc[j, 2]
            
pd_new.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648


In [80]:
pd_new.shape

(103, 5)

# PART III

## Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [81]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [82]:
toronto_data= pd_new[pd_new['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.6764,-79.293
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6796,-79.3522
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.669,-79.3156
3,M4M,East Toronto,Studio District,43.6595,-79.3409
4,M4N,Central Toronto,Lawrence Park,43.728,-79.3888
5,M4P,Central Toronto,Davisville North,43.7128,-79.3902
6,M4R,Central Toronto,North Toronto West,43.7154,-79.4057
7,M4S,Central Toronto,Davisville,43.7043,-79.3888
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.6896,-79.3832
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.6864,-79.4


### There are 38 Borough with word Toronto.

## Let's create a map of Toronto with Boroughs that cointain the word Toronto

### NOTE: PROBLEM WITH VERSION OF FOLIUM + PANDAS

In [83]:
#latitude = 43.6532
#longitude= -79.3832
# create map of TORONTO using latitude and longitude values above:
#map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
#for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
#    label = folium.Popup(label, parse_html=True)
#    folium.CircleMarker(
#        [lat, lng],
#        radius=5,
#        popup=label,
#        color='blue',
#        fill=True,
#        fill_color='#3186cc',
#        fill_opacity=0.7,
#        parse_html=False).add_to(map_toronto)  
    
#map_toronto

### Define Foursquare Credentials and Version

In [84]:
# The code was removed by Watson Studio for sharing.

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius



In [85]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [87]:
# type your answer here

#toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],latitudes=toronto_data['Latitude'],longitudes=toronto_data['Longitude'] )


#### Let's check the size of the resulting dataframe

In [90]:
#print(toronto_venues.shape)
#toronto_venues.head()

###Let's check how many venues were returned for each neighborhood

In [92]:
#toronto_venues.groupby('Neighborhood').count()

#### Let's find out how many unique categories can be curated from all the returned venues

In [93]:
#print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))